# Summary

This example getting and putting files to MinIO using reusable components.  
Credentials are set up both automatically (inferred from environment of the running container) and by hard-coding.

**WARNING:** This may overwrite any file you have in your own MinIO instance at `(standard minio tenant/)namespace/tmp/minio_get_put/(todays_date_and_time)/hello_world_*.txt` (it is timestamped at time of pipeline **compile** so likely only will overwrite something if you run the same compiled pipeline twice.  

In [1]:
# TODO:
# print_op = load_component_from_file('./print_op.yaml')

In [2]:
import kfp
from kfp import dsl, compiler
from kfp.components import load_component_from_file, load_component_from_url
import datetime

In [3]:
get_from_minio = load_component_from_file('./get_from_minio/component.yaml')
put_to_minio = load_component_from_file('./put_to_minio/component.yaml')

In [4]:
# Helper
def get_current_namespace():
    import kubernetes
    kubernetes.config.load_incluster_config()
    kube_client = kubernetes.client.CoreV1Api()
    current_namespace = open("/var/run/secrets/kubernetes.io/serviceaccount/namespace").read()
    return current_namespace


NAMESPACE = get_current_namespace()
timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
WRITE_PATH = f"{NAMESPACE}/tmp/minio_get_put/{timestamp}"

In [5]:
def pipeline():
    operations = {}

    # Get a file from shared minio location
    # Uses :
    #   * default tenant (standard)
    #   * credentials initialized from the environment (should be able to
    #     any MinIO content the namespace the pipeline is run from can access)
    # Replace source with any file you can access in MinIO
    operations['Get file with default credentials from standard'] = get_from_minio(
        source="shared/andrew-scribner/demos/minio/hello_world.txt",
    )

    # Put this file back into MinIO in a new location
    operations['Copy file to standard using default credentials'] = put_to_minio(
        source=operations['Get file with default credentials from standard'].outputs['data'],
        target=f"{WRITE_PATH}/hello_world_standard_inferred.txt",
    )
    
    # Copy with incorrect tenant but provides credentials (works)
    
    # Copy with incorrect tenant and missing a credential (does not work, fails at minio)
    
    # Copy with correct tenant, no creds (works)
    
    # copy with correct tenant, creds for a different tenant (works and writes to second tenant)

In [6]:
c = kfp.Client()

In [7]:
c.create_run_from_pipeline_func(
    pipeline,
    arguments={},
    run_name="test copy_to_minio",
    namespace='andrew-scribner',
    )

RunPipelineResult(run_id=c3412542-b175-42c2-bff1-5f10c44c2860)